In this notebook, we perform the following steps:

1. **Match Articles from Focus website**: 
   - Initially, we match 223 articles scraped from the Focus website, as detailed in the notebook **Web scraping from Focus.ipynb**, with their corresponding metadata from the Media Tenor dataset, including sentiment annotations.
   
2. **Identify and Download Missing Articles**:
   - We identify 507 articles published between 2011-2019 that were annotated by Media Tenor but were not available online and could only be found in the print version of the journal. We attempted to download them from Factiva and LexisNexis depending on their availability.

3. **Load and Match Articles from Factiva**: 
   - We begin by downloading 390 RTF files of Focus articles from Factiva. These RTF files are then converted to TXT format. After the conversion, we load the articles from the TXT files and match them with their metadata.

4. **Load and Match Articles from LexisNexis**: 
   - Similarly, we download 106 RTF files of Focus articles from LexisNexis. These RTF files are converted to TXT format, and the articles are then loaded and matched with their sentiment annotations.

5. **Combine All Articles**: 
   - Finally, we combine all these articles into one dataset and save it as a CSV file.

## Media Tenor dataset

To match the articles scraped from Focus website or downloaded from Factiva and LexisNexis with their metadata from the Media Tenor dataset, we first need to load the Media Tenor dataset. We only retain articles with non-empty titles, as it is not possible to identify and download articles without titles. Additionally, we remove one article published in 2002, which was included in the file by mistake.

In [1]:
import pandas as pd

# Load the dataset acquired from Media Tenor
sentiment_data = pd.read_csv('Daten_Wirtschaftliche_Lage.csv', encoding='utf-8', sep=';')

# Filter out rows with empty titles, as we cannot identify and download the articles without titles
sentiment_data = sentiment_data[sentiment_data['title'].notnull()]

def year(row):
    '''
    Extract the year from the 'date' column.
    '''
    # Split the date string by '.' and take the third part, then convert it to an integer
    return int(row['date'].split('.')[2])

# Apply the 'year' function to each row of the DataFrame and create a new 'year' column
# The 'axis=1' parameter indicates that the function is applied to each row
sentiment_data['year'] = sentiment_data.apply(lambda row: year(row), axis=1)

# Remove articles from the 'sentiment_data' DataFrame where the year is 2002
sentiment_data = sentiment_data[sentiment_data['year'] != 2002]

# Reset the index of the DataFrame
sentiment_data = sentiment_data.reset_index(drop=True)

# Display the first few rows of the DataFrame
sentiment_data.head()

,date,month,medium,title,topicgroup,negative,no_clear_tone,positive,Number_of_reports,AverageRating,year
0,01.01.2014,201401,WamS,Koalition,Konjunktur,0,1,0,1,0,2014
1,01.01.2017,201701,FAS,Habt bloß keine Angst vor China !,Internationale Wirtschaft,0,0,1,1,100,2017
2,01.01.2017,201701,BamS,Wir leben in einer Zeit der Wohlstands-Halluzi...,Konjunktur,0,0,1,1,100,2017
3,01.02.2015,201502,WamS,Teheran ruft,Wettbewerbsfähigkeit/Nachfrage,1,3,0,4,-25,2015
4,01.01.2017,201701,BamS,"Geht es und wirklich so gut, wie es uns Merkel...",Internationale Wirtschaft,0,1,0,1,0,2017


The titles in the Media Tenor dataset were manually entered, leading to potential inconsistencies in punctuation and spacing. To address this issue and ensure accurate matching with the titles of the articles we scrape from the website or download from databases, we normalize the titles in the dataset.

In [2]:
# Import the Normalize class from the normalize module
from normalize import Normalize

# Initialize the Normalize class with the titles from the sentiment_data DataFrame
normalizer = Normalize(sentiment_data.title)

# Apply the normalization process to the titles
normalized_titles = normalizer.normalized()

# Add the normalized titles to the sentiment_data DataFrame as a new column 'title_clean'
sentiment_data['title_clean'] = normalized_titles

sentiment_data.head()

,date,month,medium,title,topicgroup,negative,no_clear_tone,positive,Number_of_reports,AverageRating,year,title_clean
0,01.01.2014,201401,WamS,Koalition,Konjunktur,0,1,0,1,0,2014,koalition
1,01.01.2017,201701,FAS,Habt bloß keine Angst vor China !,Internationale Wirtschaft,0,0,1,1,100,2017,habt bloß keine angst vor china
2,01.01.2017,201701,BamS,Wir leben in einer Zeit der Wohlstands-Halluzi...,Konjunktur,0,0,1,1,100,2017,wir leben in einer zeit der wohlstands halluzi...
3,01.02.2015,201502,WamS,Teheran ruft,Wettbewerbsfähigkeit/Nachfrage,1,3,0,4,-25,2015,teheran ruft
4,01.01.2017,201701,BamS,"Geht es und wirklich so gut, wie es uns Merkel...",Internationale Wirtschaft,0,1,0,1,0,2017,geht es und wirklich so gut wie es uns merkel ...


We need to focus on annotated articles from Focus related to business cycle conditions, as these are the specific articles we scraped from the website or downloaded from the databases.

In [3]:
# Filter the dataset to include only articles from Focus
sentiment_data = sentiment_data[sentiment_data['medium'] == 'Focus']

# Reset the index of the DataFrame and remove the old index column
sentiment_data = sentiment_data.reset_index(drop=True)

# Further filter the dataset to include only articles related to the business cycle conditions (Konjunktur)
sentiment_data = sentiment_data[sentiment_data['topicgroup'] == 'Konjunktur']

# Reset the index of the DataFrame again and remove the old index column
sentiment_data = sentiment_data.reset_index(drop=True)

We filter the Media Tenor dataset to only keep articles where there was agreement between annotators on sentiment. Articles without annotator agreement (i.e., where `sentiment` is `NaN`) are removed.

In [4]:
from sentiment import sentiment

# Apply the 'sentiment' function to each row of the DataFrame and create a new 'sentiment' column
sentiment_data['sentiment'] = sentiment_data.apply(lambda row: sentiment(row), axis=1)

# Remove articles where there is no annotator agreement (i.e., sentiment is NaN)
sentiment_data = sentiment_data[sentiment_data['sentiment'].notnull()]

# Reset the index of the DataFrame again and remove the old index column
sentiment_data = sentiment_data.reset_index(drop=True)

# Display the first few rows of the filtered DataFrame to verify the results
sentiment_data.head()

,date,month,medium,title,topicgroup,negative,no_clear_tone,positive,Number_of_reports,AverageRating,year,title_clean,sentiment
0,01.07.2017,201707,Focus,Läuft.,Konjunktur,0,0,1,1,100,2017,läuft,1.0
1,01.04.2017,201704,Focus,Anlegen in Zeiten von Trump,Konjunktur,0,0,2,2,100,2017,anlegen in zeiten von trump,1.0
2,01.07.2013,201307,Focus,Gewinne trotz Wackelbörse,Konjunktur,0,0,3,3,100,2013,gewinne trotz wackelbörse,1.0
3,01.10.2011,201110,Focus,Macht Europa nicht kaputt,Konjunktur,0,1,0,1,0,2011,macht europa nicht kaputt,0.0
4,01.10.2012,201210,Focus,Der Kandidat sucht aus,Konjunktur,0,1,0,1,0,2012,der kandidat sucht aus,0.0


## Match Articles from Focus website

Next, we load the articles that we scraped from Focus website and saved in the file `focus_2011_2019.csv`. These articles are read into a DataFrame named `focus`.

In [5]:
import os

# Define the path for a CSV file
path_focus = os.path.join(os.getcwd(), 'MediaTenor_LexisNexis_Factiva', 'focus_2011_2019.csv')

# Load the articles scraped from Focus website
focus = pd.read_csv(path_focus, encoding='utf-8', sep=';', names=["date", "text", "title", "title_clean"])

# Reset the index of the DataFrame
focus = focus.reset_index(drop=True)

# Display the first few rows of the DataFrame
focus.head()

,date,text,title,title_clean
0,15.11.2013,NOTIZEN AUS DER WIRTSCHAFT. Der Favoritenwechs...,NOTIZEN AUS DER WIRTSCHAFT,notizen aus der wirtschaft
1,15.11.2013,MONTAG IST ZEUGNISTAG. WOLFGANG MAYRHUBER Mill...,MONTAG IST ZEUGNISTAG,montag ist zeugnistag
2,15.11.2013,NOTIZEN AUS DER WIRTSCHAFT. Die Grafit-Aktie ....,NOTIZEN AUS DER WIRTSCHAFT,notizen aus der wirtschaft
3,15.11.2013,NOTIZEN AUS DER WIRTSCHAFT. Der japanische Yen...,NOTIZEN AUS DER WIRTSCHAFT,notizen aus der wirtschaft
4,15.11.2013,MONTAG IST ZEUGNISTAG. BIRGIT FISCHER Insiderw...,MONTAG IST ZEUGNISTAG,montag ist zeugnistag


Now we merge these articles with their metadata from the Media Tenor dataset. We start by performing an initial merge based on the exact `title_clean` and `date`. If an exact match is not found, we adjust the publication date of the scraped articles by one or two days forward and backward, and attempt to match again. This adjustment is necessary because Media Tenor annotated articles from the print version of Focus, while the online version may have the same content but slightly different publication dates. By accounting for this possible discrepancy, we ensure accurate matching. The final merged DataFrame includes columns for the journal's name, publication date (day, month, and year), article title, text, sentiment, and file name.

In [6]:
from datetime import datetime

def adjust_date(date_str, delta):
    
    '''Function to adjust the publication date'''
    
    # Convert the date string into a datetime object
    date = datetime.strptime(date_str, '%d.%m.%Y')
    # Adjust the date by the specified number of days (delta)
    adjusted_date = date + pd.Timedelta(days=delta)
    
    # Convert the adjusted datetime object back into a string in the original format
    return adjusted_date.strftime('%d.%m.%Y')

# Initial matching based on exact date
data_match_scraped = pd.merge(sentiment_data, focus, how='inner', on=['title_clean', 'date'])

# Date adjustments: iterate through the delta values
for delta in [1, -1, 2, -2]:
    # Adjust dates in the 'focus' DataFrame
    focus['adjusted_date'] = focus['date'].apply(lambda date: adjust_date(date, delta))
    # Perform the merge with the adjusted dates
    matched_data = pd.merge(sentiment_data, focus, how='inner', left_on=['title_clean', 'date'], right_on=['title_clean', 'adjusted_date'])
    # Rename the 'date_x' column to 'date'
    matched_data = matched_data.rename(columns={'date_x': 'date'})
    # Drop the 'date_y' and 'adjusted_date' columns
    matched_data = matched_data.drop(columns=['date_y', 'adjusted_date'])
    # Append the new matches to the main DataFrame
    data_match_scraped = pd.concat([data_match_scraped, matched_data], ignore_index=True)
    
# Remove duplicates
data_match_scraped = data_match_scraped.drop_duplicates(['date', 'title_clean'])

# Rename columns and split date
# Rename the 'medium' column to 'journal'
data_match_scraped = data_match_scraped.rename(columns={'medium': 'journal'})
# Split 'date' into 'day', 'month', and 'year'
data_match_scraped['date'] = pd.to_datetime(data_match_scraped['date'], format='%d.%m.%Y')
data_match_scraped['day'] = data_match_scraped['date'].dt.day
data_match_scraped['month'] = data_match_scraped['date'].dt.month
data_match_scraped['year'] = data_match_scraped['date'].dt.year
# Create 'file' column that contains the name of the CSV file
data_match_scraped['file'] = 'focus_2011_2019.csv'
# Rename the 'title_y' column to 'title' to reflect the title from the Focus dataset
data_match_scraped = data_match_scraped.rename(columns={'title_y': 'title'})
# Convert the datetime object back into a string in the original format
data_match_scraped['date'] = data_match_scraped['date'].dt.strftime('%d.%m.%Y')
# Reorder columns
columns_order = ['journal', 'day', 'month', 'year', 'title', 'text', 'sentiment', 'file', 'title_clean', 'date']
data_match_scraped = data_match_scraped[columns_order]

# Sort data chronologically and reset index
data_match_scraped = data_match_scraped.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
data_match_scraped = data_match_scraped.reset_index(drop=True)

# Display the first few rows
data_match_scraped.head()

,journal,day,month,year,title,text,sentiment,file,title_clean,date
0,Focus,18,11,2013,Die große Koalition der Geldausgeber,Die große Koalition der Geldausgeber. Mehr Job...,1.0,focus_2011_2019.csv,die große koalition der geldausgeber,18.11.2013
1,Focus,18,11,2013,Zahlen aus der Wirtschaft,"Zahlen aus der Wirtschaft. Auf 1,20 US-Dollar ...",1.0,focus_2011_2019.csv,zahlen aus der wirtschaft,18.11.2013
2,Focus,18,11,2013,"""Das erinnert mich an Weihnachten""","""Das erinnert mich an Weihnachten"" Der Wirtsch...",1.0,focus_2011_2019.csv,das erinnert mich an weihnachten,18.11.2013
3,Focus,25,11,2013,"""Abgerechnet wird am Schluss""","""Abgerechnet wird am Schluss"" Der stellvertret...",1.0,focus_2011_2019.csv,abgerechnet wird am schluss,25.11.2013
4,Focus,25,11,2013,Vor neuem Höhenflug?,Vor neuem Höhenflug? Der japanische Nikkei-Ind...,1.0,focus_2011_2019.csv,vor neuem höhenflug,25.11.2013


Next, we remove 14 matched articles with publication dates and titles listed in `pairs_to_remove` as they were accidentally downloaded twice. These articles were both scraped from the website and downloaded from LexisNexis or Factiva. Since the versions from the databases generally have better formatting, we prefer them. Therefore, we drop the 14 duplicates that were scraped from the website. As a final result, we successfully scraped **223** articles from the Focus website and matched them with their sentiment annotations.

In [7]:
# List of specific title and date pairs to be removed
pairs_to_remove = [
    ('alle drei minuten ein angriff', '02.06.2014'),
    ('eine moralische krise', '02.06.2014'),
    ('die meister des idealen strahls', '02.06.2014'),
    ('mehr können wir uns nicht leisten', '02.06.2014'),
    ('was wenn die wirtschaft kaum noch wächst', '02.06.2014'),
    ('wer gewinnt wenn sich europa erholt', '12.09.2015'),
    ('die schönen aktien und ihre klugen händler', '02.04.2016'),
    ('kurstreiber maschinenbau', '02.04.2016'),
    ('europa hat harte zeiten vor sich', '17.06.2017'),
    ('wer das bier bestellt soll es auch zahlen', '01.09.2018'),
    ('veränderungen tun weh zittern ist nicht erlaubt', '01.12.2018'),
    ('schutz gegen kursverluste', '01.12.2018'),
    ('wir können dagegen ankämpfen', '25.05.2019'),
    ('wer nächste woche wichtig wird', '25.05.2019')
]

# Convert list of pairs to a DataFrame
pairs_df = pd.DataFrame(pairs_to_remove, columns=['title_clean', 'date'])

# Merge the 'data_match_scraped' with 'pairs_df' to identify rows to be removed
merged_df = data_match_scraped.merge(pairs_df, on=['title_clean', 'date'], how='left', indicator=True)

# Keep rows that do not appear in the pairs_df
data_match_scraped = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# Drop the 'date' column
data_match_scraped = data_match_scraped.drop(columns=['date'])

# Reset the index of the DataFrame
data_match_scraped = data_match_scraped.reset_index(drop=True)

# Sort data chronologically and reset index
data_match_scraped = data_match_scraped.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
data_match_scraped = data_match_scraped.reset_index(drop=True)

# Display the number of matched articles and the first few rows
print(f"Number of matched articles: {len(data_match_scraped)}")
data_match_scraped.head()

Number of matched articles: 223


,journal,day,month,year,title,text,sentiment,file,title_clean
0,Focus,18,11,2013,Die große Koalition der Geldausgeber,Die große Koalition der Geldausgeber. Mehr Job...,1.0,focus_2011_2019.csv,die große koalition der geldausgeber
1,Focus,18,11,2013,Zahlen aus der Wirtschaft,"Zahlen aus der Wirtschaft. Auf 1,20 US-Dollar ...",1.0,focus_2011_2019.csv,zahlen aus der wirtschaft
2,Focus,18,11,2013,"""Das erinnert mich an Weihnachten""","""Das erinnert mich an Weihnachten"" Der Wirtsch...",1.0,focus_2011_2019.csv,das erinnert mich an weihnachten
3,Focus,25,11,2013,"""Abgerechnet wird am Schluss""","""Abgerechnet wird am Schluss"" Der stellvertret...",1.0,focus_2011_2019.csv,abgerechnet wird am schluss
4,Focus,25,11,2013,Vor neuem Höhenflug?,Vor neuem Höhenflug? Der japanische Nikkei-Ind...,1.0,focus_2011_2019.csv,vor neuem höhenflug


## Identify and Download Missing Articles

The following code identifies **507** articles published between 2011-2019 that were annotated by Media Tenor but were not available online and could only be found in the print version of the journal. Since we were unable to scrape these articles from the Focus website, we identified these missing articles and then attempted to download them from Factiva and LexisNexis depending on their availability.

In [8]:
# Create the date string in 'data_match_scraped' DataFrame
data_match_scraped['day_str'] = data_match_scraped['day'].astype(str).str.zfill(2)
data_match_scraped['month_str'] = data_match_scraped['month'].astype(str).str.zfill(2)
data_match_scraped['year_str'] = data_match_scraped['year'].astype(str)
data_match_scraped['date'] = data_match_scraped['day_str'] + '.' + data_match_scraped['month_str'] + '.' + data_match_scraped['year_str']

# Create tuples of (title_clean, date) for comparison
sentiment_data_tuples = sentiment_data[['title_clean', 'date']].apply(tuple, axis=1)
data_match_scraped_tuples = data_match_scraped[['title_clean', 'date']].apply(tuple, axis=1)

# Identify articles that still need to be downloaded
to_download = sentiment_data[~sentiment_data_tuples.isin(data_match_scraped_tuples)]

# Reset the index of the DataFrame and remove the old index column
to_download = to_download.reset_index(drop=True)

# Convert the 'date' column to datetime format for accurate sorting
to_download['date'] = pd.to_datetime(to_download['date'], format='%d.%m.%Y')

# Sort 'to_download' based on 'date'
to_download = to_download.sort_values(by='date')

# Reset the index of the DataFrame and remove the old index column
to_download = to_download.reset_index(drop=True)

# Save the result to a CSV file
to_download.to_csv('to_download_focus.csv', encoding='utf-8-sig', sep=',')

# Drop the temporary columns
data_match_scraped = data_match_scraped.drop(columns=['day_str', 'month_str', 'year_str', 'date'])

print(f"Number of articles to download: {len(to_download)}")

# Display the first few rows of the DataFrame to verify the result
to_download.head()

Number of articles to download: 507


,date,month,medium,title,topicgroup,negative,no_clear_tone,positive,Number_of_reports,AverageRating,year,title_clean,sentiment
0,2011-01-03,201101,Focus,7 Aktien- & Fonds-Trends,Konjunktur,0,0,9,9,100,2011,7 aktien fonds trends,1.0
1,2011-01-17,201101,Focus,Frisches Geld für Aktionäre,Konjunktur,0,0,1,1,100,2011,frisches geld für aktionäre,1.0
2,2011-02-07,201102,Focus,Das Inflationsgefühl täuscht nicht,Konjunktur,0,2,0,2,0,2011,das inflationsgefühl täuscht nicht,0.0
3,2011-03-14,201103,Focus,Japan erschüttert die Welt,Konjunktur,0,3,0,3,0,2011,japan erschüttert die welt,0.0
4,2011-03-14,201103,Focus,keine 5 Zeilen (nur Tabellennennung),Konjunktur,0,0,1,1,100,2011,keine 5 zeilen nur tabellennennung,1.0


## Load and Match Articles from Factiva

Next, we focus on loading Focus articles downloaded from Factiva and matching them with their metadata. In our first step, we convert the RTF files into TXT format. All the RTF files are stored in `MediaTenor_LexisNexis_Factiva/Focus_Konjunktur_Factiva_rtf`. The converted TXT files are stored in `MediaTenor_LexisNexis_Factiva/Focus_Konjunktur_Factiva_txt`.

In [9]:
# Import the function for converting RTF to TXT
from convert_rtf_to_txt import convert_rtf_to_txt

# Define paths for Focus RTF and TXT directories
focus_rtf_path = os.path.join(os.getcwd(), 'MediaTenor_LexisNexis_Factiva', 'Focus_Konjunktur_Factiva_rtf')
focus_txt_path = os.path.join(os.getcwd(), 'MediaTenor_LexisNexis_Factiva', 'Focus_Konjunktur_Factiva_txt')

# Convert RTF files to TXT format for Focus
convert_rtf_to_txt(focus_rtf_path, focus_txt_path)

As soon as the RTF files were transformed into TXT format, we made a few changes to the TXT files. Specifically, we corrected several titles to ensure accurate spelling and punctuation, which is important for matching them with the metadata from the Media Tenor dataset. For example:
- "Kalter Krieg **2 . 0**" was corrected to "Kalter Krieg **2.0**"
- "Die **Rohstoff- Formel** fürs Depot" was corrected to "Die **Rohstoff-Formel** fürs Depot"

Additionally, a few articles were compilations of multiple pieces. In such cases, we manually selected the annotated article from the compilation.

Once the TXT files were ready, we used the function `extract_article_data_focus_factiva` to load the text of the articles along with the journal's name, date of publication, title, and file name into a dictionary called `article_data`.

In [10]:
import extract_article_data_focus_factiva

# Read and extract relevant information from TXT files in Focus directory.
article_data = extract_article_data_focus_factiva.extract_article_data_focus_factiva(focus_txt_path)

We use the `article_data` dictionary to create a DataFrame `focus_factiva` that includes columns for the journal's name, publication date (day, month, and year), article title, text, and file name.

In [11]:
# Create a DataFrame from the collected data
focus_factiva = pd.DataFrame({
    'journal': article_data['journal'],
    'day': article_data['day'],
    'month': article_data['month'],
    'year': article_data['year'],
    'title': article_data['title'],
    'text': article_data['text'],
    'file': article_data['file']
})

focus_factiva.head()

,journal,day,month,year,title,text,file
0,Focus,15,Juni,2019,Die Klimakrise ist unser dritter Weltkrieg,"Die USA können es sich nicht leisten, auf den ...",Factiva-20200827-1230.txt
1,Focus,25,Mai,2019,Wer nächste Woche wichtig wird,Der Terminkalender vom 26. bis 31. Mai. So. Eu...,Factiva-20200827-1234.txt
2,Focus,25,Mai,2019,Kalter Krieg 2.0,"US-Präsident Donald Trump drängt Google, alle ...",Factiva-20200827-1237.txt
3,Focus,25,Mai,2019,Die Crashtest - Dummys der Wall Street,"Es deutet wenig darauf hin, dass die Welt viel...",Factiva-20200827-1240.txt
4,Focus,11,Mai,2019,Die Billionenkette,Das Blockchain-System wurde durch Kryptowährun...,Factiva-20200827-1241 (1).txt


To match the full texts of the loaded articles with their sentiment annotations from the Media Tenor dataset, we follow several key steps. First, we create a date in the same format as in the `sentiment_data` DataFrame. Next, we normalize the titles to ensure accurate matching. We also verify that there are no duplicate articles. After pre-processing, we merge the articles loaded from Factiva with their sentiment annotations from the Media Tenor dataset. We then sort the final DataFrame `data_match_factiva` in chronological order and retain only the relevant columns. Through this process, we successfully matched **390** Focus articles from Factiva with their sentiment annotations.

In [12]:
# Create dictionary to transform month name into month number
name_to_number = {
    u'Januar': '01', u'Februar': '02', u'M\xe4rz': '03', u'April': '04', u'Mai': '05',
    u'Juni': '06', u'Juli': '07', u'August': '08', u'September': '09', u'Oktober': '10',
    u'November': '11', u'Dezember': '12'
}

# Transform month names into month numbers
focus_factiva['month_num'] = focus_factiva['month'].map(name_to_number)

# Create dictionary to transform single-digit day numbers
day_transform = {u'1': '01', u'2': '02', u'3': '03', u'4': '04', u'5': '05', u'6': '06', u'7': '07', u'8': '08', u'9': '09'}

# Transform single-digit day numbers into two-digit format
focus_factiva['day'] = focus_factiva['day'].map(lambda d: day_transform.get(d, d))

# Combine day, month, and year into a date string
focus_factiva['date'] = focus_factiva.apply(lambda row: f"{row['day']}.{row['month_num']}.{row['year']}", axis=1)

# Reset the index of the DataFrame
focus_factiva = focus_factiva.reset_index(drop=True)

# Initialize the Normalize class with the titles from the focus_factiva DataFrame
normalizer = Normalize(focus_factiva.title)

# Apply the normalization process to the titles
normalized_titles = normalizer.normalized()

# Add the normalized titles to the focus_factiva DataFrame as a new column 'title_clean'
focus_factiva['title_clean'] = normalized_titles

# Merge with sentiment_data on title_clean and date
data_match_factiva = pd.merge(sentiment_data, focus_factiva, how='inner', on=['title_clean', 'date'])

# Rename the 'month_num' column to 'month'
data_match_factiva = data_match_factiva.rename(columns={'month_num': 'month'})

# Rename the 'year_y' column to 'year'
data_match_factiva = data_match_factiva.rename(columns={'year_y': 'year'})

# Convert year, month, and day to integers
data_match_factiva['year'] = data_match_factiva['year'].astype(int)
data_match_factiva['month'] = data_match_factiva['month'].astype(int)
data_match_factiva['day'] = data_match_factiva['day'].astype(int)

# Sort the data in chronological order
data_match_factiva = data_match_factiva.sort_values(['year', 'month', 'day'], ascending=[True, True, True])

# Reset the index of the DataFrame
data_match_factiva = data_match_factiva.reset_index(drop=True)

# Rename the 'title_y' column to 'title' to reflect the title from the Factiva dataset
data_match_factiva = data_match_factiva.rename(columns={'title_y': 'title'})

# Select only the required columns
data_match_factiva = data_match_factiva[['journal', 'day', 'month', 'year', 'title', 'text', 'sentiment', 'file', 'title_clean']]

# Print the number of articles from Factiva
num_factiva_articles = len(data_match_factiva)

print(f"Number of articles from Factiva: {num_factiva_articles}")

# Display the first few rows of the final matched dataset
data_match_factiva.head()

Number of articles from Factiva: 390


,journal,day,month,year,title,text,sentiment,file,title_clean
0,Focus,30,7,2012,Raus oder aus,Muss Griechenland aus dem Euro aussteigen? Geh...,-1.0,Factiva-20200828-1232.txt,raus oder aus
1,Focus,30,7,2012,MONTAG IST ZEUGNISTAG,RÜDIGER GRUBE. Die Bahn rollt - und das besser...,0.0,Factiva-20200828-1231 (1).txt,montag ist zeugnistag
2,Focus,6,8,2012,Kirchhof-Comeback,ALTSCHULDEN ABBAUEN Der frühere Verfassungsric...,0.0,Factiva-20200828-1231.txt,kirchhof comeback
3,Focus,6,8,2012,» Ohne die FDP hätten wir längst Euro - Bonds «,Außenminister Guido Westerwelle (FDP) sieht zu...,1.0,Factiva-20200828-1230 (1).txt,ohne die fdp hätten wir längst euro bonds
4,Focus,6,8,2012,Der Traum vom ewigen Urlaub,Ob eine Stunde von zu Hause oder gleich Mallor...,0.0,Factiva-20200828-1230.txt,der traum vom ewigen urlaub


## Load and Match Articles from LexisNexis

In this section, we aim to load articles from Focus that were downloaded from LexisNexis and match them with their sentiment annotations. We begin by converting the RTF files into TXT format. The original RTF files are located in `MediaTenor_LexisNexis_Factiva/Focus_Konjunktur_LexisNexis_rtf`, and the resulting TXT files are stored in `MediaTenor_LexisNexis_Factiva/Focus_Konjunktur_LexisNexis_txt`.

In [13]:
# Define paths for Focus RTF and TXT directories
focus_lexisnexis_rtf_path = os.path.join(os.getcwd(), 'MediaTenor_LexisNexis_Factiva', 'Focus_Konjunktur_LexisNexis_rtf')
focus_lexisnexis_txt_path = os.path.join(os.getcwd(), 'MediaTenor_LexisNexis_Factiva', 'Focus_Konjunktur_LexisNexis_txt')

# Convert RTF files to TXT format for Focus
convert_rtf_to_txt(focus_lexisnexis_rtf_path, focus_lexisnexis_txt_path)

Once the RTF files were converted to TXT format, we made some adjustments. Before extracting titles from the 'Search Terms' in our TXT files, we corrected a few titles to ensure they matched those in the Media Tenor dataset. For example, "**´Krise?** Welche Krise?" was corrected to "**Krise?** Welche Krise?" to ensure accurate matching.

Additionally, for some documents, we removed content found at the end of the text that was not part of the main article, such as source information.

After preparing the TXT files, we used the `extract_article_data_focus_lexisnexis` function to load the articles' text, along with the journal name, publication date, title, and file name, into a dictionary called a`article_data_lexisnexis`.

In [14]:
import extract_article_data_focus_lexisnexis

# Read and extract relevant information from TXT files in Focus directory.
article_data_lexisnexis = extract_article_data_focus_lexisnexis.extract_article_data_focus_lexisnexis(focus_lexisnexis_txt_path)

We use the `article_data_lexisnexis` dictionary to create a DataFrame `focus_lexisnexis` that includes columns for the journal's name, publication date (day, month, and year), article title, text, and file name.

In [15]:
# Create a DataFrame from the collected data
focus_lexisnexis = pd.DataFrame({
    'journal': article_data_lexisnexis['journal'],
    'day': article_data_lexisnexis['day'],
    'month': article_data_lexisnexis['month'],
    'year': article_data_lexisnexis['year'],
    'title': article_data_lexisnexis['title'],
    'text': article_data_lexisnexis['text'],
    'file': article_data_lexisnexis['file']
})

focus_lexisnexis.head()

,journal,day,month,year,title,text,file
0,Focus,25,Juni,2012,Solides Skandinavien,Zinsjäger können bereits mit überschaubaren Ei...,30 Tipps _ Anleihen und Anleihefonds_ Gesamtti...
1,Focus,19,Dezember,2011,Abgrenzung von den linken Parteien,NEUAUSRICHTUNG Mit einem klaren politischen Ab...,Abgrenzung von den linken Parteien.txt
2,Focus,21,Mai,2012,Abrechnung mit dem Euro,Bestsellerautor Thilo Sarrazin analysiert die ...,Abrechnung mit dem Euro.txt
3,Focus,27,Dezember,2011,Boom statt kollaps?,Eine Grundschule im Berliner Westen. Generalpr...,Boom statt Kollaps_.txt
4,Focus,8,August,2011,Brasilien & Co,SCHWELLENLÄNDER Aktienanleger brauchen derzeit...,Brasilien & Co._ Chancen trotz Panik.txt


To match the full texts of the loaded articles with their sentiment annotations from the Media Tenor dataset, we follow several key steps. First, we create a date in the same format as in the `sentiment_data` DataFrame. Next, we normalize the titles to ensure accurate matching. We also verify that there are no duplicate articles. After pre-processing, we merge the articles loaded from LexisNexis with their sentiment annotations from the Media Tenor dataset. We then sort the final DataFrame `data_match_lexisnexis` in chronological order and retain only the relevant columns. Through this process, we successfully matched **106** Focus articles from LexisNexis with their sentiment annotations.

In [16]:
# Transform month names into month numbers
focus_lexisnexis['month_num'] = focus_lexisnexis['month'].map(name_to_number)

# Transform single-digit day numbers into two-digit format
focus_lexisnexis['day'] = focus_lexisnexis['day'].map(lambda d: day_transform.get(d, d))

# Combine day, month, and year into a date string
focus_lexisnexis['date'] = focus_lexisnexis.apply(lambda row: f"{row['day']}.{row['month_num']}.{row['year']}", axis=1)

# Initialize the Normalize class with the titles from the spiegel_lexisnexis DataFrame
normalizer = Normalize(focus_lexisnexis.title)

# Apply the normalization process to the titles
normalized_titles = normalizer.normalized()

# Add the normalized titles to the 'focus_lexisnexis' DataFrame as a new column 'title_clean'
focus_lexisnexis['title_clean'] = normalized_titles

# Merge with sentiment_data on title_clean and date
data_match_lexisnexis = pd.merge(sentiment_data, focus_lexisnexis, how='inner', on=['title_clean', 'date'])

# Rename the 'month_num' column to 'month'
data_match_lexisnexis = data_match_lexisnexis.rename(columns={'month_num': 'month'})

# Rename the 'year_y' column to 'year'
data_match_lexisnexis = data_match_lexisnexis.rename(columns={'year_y': 'year'})

# Convert year, month, and day to integers
data_match_lexisnexis['year'] = data_match_lexisnexis['year'].astype(int)
data_match_lexisnexis['month'] = data_match_lexisnexis['month'].astype(int)
data_match_lexisnexis['day'] = data_match_lexisnexis['day'].astype(int)

# Sort the data in chronological order
data_match_lexisnexis = data_match_lexisnexis.sort_values(['year', 'month', 'day'], ascending=[True, True, True])

# Reset the index of the DataFrame
data_match_lexisnexis = data_match_lexisnexis.reset_index(drop=True)

# Rename the 'title_y' column to 'title' to reflect the title from the LexisNexis dataset
data_match_lexisnexis = data_match_lexisnexis.rename(columns={'title_y': 'title'})

# Select only the required columns
data_match_lexisnexis = data_match_lexisnexis[['journal', 'day', 'month', 'year', 'title', 'text', 'sentiment', 'file', 'title_clean']]

# Print the number of articles from LexisNexis
num_lexisnexis_articles = len(data_match_lexisnexis)

print(f"Number of articles from LexisNexis: {num_lexisnexis_articles}")

# Display the last few rows of the final matched dataset
data_match_lexisnexis.tail()

Number of articles from LexisNexis: 106


,journal,day,month,year,title,text,sentiment,file,title_clean
101,Focus,10,2,2018,Wo Deutschland am stärksten ist,Im Gesamtranking sichert sich der Landkreis Mü...,1.0,Wo Deutschland am st_rksten ist.txt,wo deutschland am stärksten ist
102,Focus,10,2,2018,Die Verlierer-Regierer,Mit letzter Kraft schleppen sich Union und SPD...,1.0,Die Verlierer- Regierer.txt,die verlierer regierer
103,Focus,5,5,2018,Ein Moment Ewigkeit,Eben noch flogen Beschimpfungen und Drohungen ...,1.0,Panmunjom_ Ein Moment Ewigkeit.txt,ein moment ewigkeit
104,Focus,9,6,2018,"Deutschland entscheidet, ob der Euro überlebt",Als Finanzminister Griechenlands lehnte Yanis ...,-1.0,Deutschland entscheidet_ ob der Euro _berlebt.txt,deutschland entscheidet ob der euro überlebt
105,Focus,1,12,2018,"Veränderungen tun weh, zittern ist nicht erlaubt","Von Robert Schneider, Chefredakteur. Liebe Les...",-1.0,Ver_nderungen tun weh_ zittern ist nicht erlau...,veränderungen tun weh zittern ist nicht erlaubt


## Combine All Articles

As the final step, we consolidate all Focus articles, including those that were scraped and those downloaded from Factiva and LexisNexis, into a single DataFrame called `focus_all`. This combined DataFrame is then saved as a CSV file named `focus.csv`.

In [17]:
# Combine all articles from scraped, Factiva, and LexisNexis datasets into a single DataFrame
focus_all = pd.concat([data_match_scraped, data_match_factiva, data_match_lexisnexis], sort=False)

# Reset the index of the combined DataFrame
focus_all = focus_all.reset_index(drop=True)

# Print the total number of articles in the combined DataFrame
total_articles = len(focus_all)
print(f"Total number of articles: {total_articles}")

# Display the first few rows of the combined DataFrame to verify the merge
focus_all.head()

Total number of articles: 719


,journal,day,month,year,title,text,sentiment,file,title_clean
0,Focus,18,11,2013,Die große Koalition der Geldausgeber,Die große Koalition der Geldausgeber. Mehr Job...,1.0,focus_2011_2019.csv,die große koalition der geldausgeber
1,Focus,18,11,2013,Zahlen aus der Wirtschaft,"Zahlen aus der Wirtschaft. Auf 1,20 US-Dollar ...",1.0,focus_2011_2019.csv,zahlen aus der wirtschaft
2,Focus,18,11,2013,"""Das erinnert mich an Weihnachten""","""Das erinnert mich an Weihnachten"" Der Wirtsch...",1.0,focus_2011_2019.csv,das erinnert mich an weihnachten
3,Focus,25,11,2013,"""Abgerechnet wird am Schluss""","""Abgerechnet wird am Schluss"" Der stellvertret...",1.0,focus_2011_2019.csv,abgerechnet wird am schluss
4,Focus,25,11,2013,Vor neuem Höhenflug?,Vor neuem Höhenflug? Der japanische Nikkei-Ind...,1.0,focus_2011_2019.csv,vor neuem höhenflug


In [18]:
# Sort the combined DataFrame in chronological order
focus_all = focus_all.sort_values(['year', 'month', 'day'], ascending=[True, True, True])

# Reset the index of the DataFrame
focus_all = focus_all.reset_index(drop=True)

# Drop the 'title_clean' column as it is no longer needed
focus_all = focus_all.drop(columns=['title_clean'])

# Save the combined DataFrame to a CSV file
focus_all.to_csv('focus.csv', encoding='utf-8-sig', sep=';')